# i.MX943 SRK Revocation Example

This Jupyter Notebook provides guidance on how to perform SRK revocation on ELE based i.MX devices using SPSDK.

This example uses a signed U-Boot binary targeting i.MX943. To build and sign a bootable image with ELE firmware and U-Boot bootloader, follow the procedure described in the i.MX95 signed AHAB with U-Boot example.
 

## 1. Prerequisites
 - SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the installation documentation.)
 - This demo was tested with i.MX943 EVK board wih LPDDR4 memory. 

### 1.1 Requirements
 - A valid signed bootable image containing U-Boot, ELE firmware and other necessary binaries for i.MX943 (see [i.MX95 signed AHAB with U-BOOT](../imx95/imx95_signed_ahab_uboot.ipynb) examples).
 - The OEM Super Root Key Hash(SRKH) should already be programmed in the fuses. The referenced example includes the steps required to complete this procedure. The device is in OEM Open lifecycle.


## 2. SRK revocation

A SRK table consists of 1 to 4 SRKs. A revocation procedure may be performed if one or more are compromised. After revoking a SRK, the next available key in the table may be used for image signing.

If the SRK revoke bit is set in the AHAB container header, the container must first be successfully authenticated. Only then can the *commit* command be issued to the ELE, which programs the corresponding fuse bit. After this step, the revoked SRK(s) can no longer be used to authenticate any image.

### 2.1 Set the SRK revoke mask

Before generating the signed AHAB bootable image, ensure that the *srk_revoke_mask* field is set identically accross all AHAB container YAML files.

If SRK1 needs to be revoked, set the *srk_revoke_mask* field to 0x1. Multiple SRK keys can be revoked simultaneously. For example, to revoke both SRK0 and SRK1, use a *srk_revoke_mask* value of 0x3.

### 2.2 Download image
Set the boot mode to Cortex-M Serial Downloader and download the files using the UUU tool.

```bash
nxpuuu write -f mimx943 -b emmc signed_flash.bin
```

Now change the boot mode to Cortex-M eMMC and reset the board. Find the serial port that belongs to U-Boot console and interrupt the boot process. 
When the console is switched to the U-Boot menu, we can use the *nxpele* tool to communicate with the ELE and check the logged events. There should be no AHAB events occurring at this point.

```bash
nxpele -f mimx943 -d uboot_serial -p /dev/ttyUSB2 get-events
```

### 2.3 Perform the SRK revocation

This can be done using the *nxpele* tool.

**Note that this step is irreversible and the revoked SRK(s) will become unusable.**

```bash
nxpele -f mimx943 -d uboot_serial -p /dev/ttyUSB2 commit -i oem_srk_revocation
```


## 3. Test 

Booting a device with an image signed using a revoked key will trigger AHAB events. These events can be retrieved using *nxpele*.

```bash
nxpele -f mimx943 -d uboot_serial -p /dev/ttyUSB2 get-events
```


Attempting to sign images using revoked keys will trigger errors. Refer to the log output below for more details.

```bash
nxpimage -v bootable-image export -c bootimg_mimx943_serial_downloader.yaml -o signed_flash.bin

SPSDKValueError: SPSDK: Cannot export AHAB container from the configuration:
SPSDK: ==================================================== AHAB (Error) ====================================================
AHAB Image:
Family: mimx943, Revision: latest
Target memory: standard
Max cont. count: 3 Max image. count: 8 Containers count: 3
======================================================================================================================
=============================================== Container 2 (Error) ================================================
AHAB Container:
SRK Set: oem. Signed by OEM keys
Offset: 0x8000
Flags: 0x612
Fuse version: 0x0
SW version: 0x0
Images count: 7

This is signed container, check that the SRK hash fuses has following values:

--== Grouped register name: SRKH ==--
OTP ID: 608, Value: 0x30DEEA92
OTP ID: 609, Value: 0x8BB373CD
OTP ID: 610, Value: 0x998C7136
OTP ID: 611, Value: 0x5FE1B26B
OTP ID: 612, Value: 0xFB007B25
OTP ID: 613, Value: 0xC6533731
OTP ID: 614, Value: 0xD426EF8A
OTP ID: 615, Value: 0xED6C7BB1
OTP ID: 616, Value: 0x4E090592
OTP ID: 617, Value: 0x6E7A5F04
OTP ID: 618, Value: 0x77D3B86B
OTP ID: 619, Value: 0x45E432C9
OTP ID: 620, Value: 0x994DF0E7
OTP ID: 621, Value: 0x6A659D5B
OTP ID: 622, Value: 0x3A7D5A3F
OTP ID: 623, Value: 0xD8EB7801

====================================================================================================================
Container signing(Error)
Used SRK key ID(Error): SRK ID 1 is revoked
```